In [4]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [2]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")

In [ ]:
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

In [4]:
companies_path = 'CSVs/clutch_companies.csv'

In [5]:
# Clutch IT Firms URL
clutch_url = 'https://clutch.co/it-services/outsourcing' 

In [6]:
#HTML Div class references (not using yet)

firms_number_html='tabs-info'
firms_list_html='company_logotype'
firm_name_html='header-company--title'
firm_score_html='ratingValue'
firm_location_html='location-name'
firm_clients_html='field-name-clients'
firm_client_name='field c'

In [7]:
def get_companies_csv():
    try: return pd.read_csv(companies_path)
    except: return pd.DataFrame()

In [8]:
def get_total_pages_at(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    firms_tab_text = soup.find("div",attrs={"class" : "tabs-info"}, recursive=True).text
    firms_number = firms_tab_text.replace(',', '').replace('Firms', '').strip()
    return math.ceil(int(firms_number) / 19)

In [9]:
def find_url_references_at(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    url_list = soup.find_all("div",attrs={"class" : "company_logotype"}, recursive=True)
    return [url.find('a')['href'] for url in url_list]

In [10]:
def collect_data_from(ref_list):
    df = get_companies_csv()
    for ref in ref_list:
        print('Collecting data from:', ref)
        data = colect_infos_at(ref)
        df = df.append(data, ignore_index=True)
    return df

In [11]:
def colect_infos_at(reference):
    browser.get(f"https://clutch.co{reference}")
    time.sleep(2)
    
    soup = BeautifulSoup(browser.page_source, features='html.parser')

    try:
        name = soup.find("h1", attrs={"class" : "header-company--title"}).text.replace('\n', '').strip()
    except:
        name = 'not found'
    
    try:
        score = soup.find("span", attrs={"itemprop" : "ratingValue"}).text.replace('\n', '').strip()
    except:
        score = 'not found' 
        
    try:
        location = soup.find("span", attrs={"class" : "location-name"}).text.replace('\n', '').strip()
    except:
        location = 'not found'
        
    try:
        main_div = soup.find("div", attrs={"class" : "module-list"}) 
        main_div.find_all("span")
        main_div = [i.text for i in main_div.find_all("span")]
        
        valuation = main_div[0]
        price_hour = main_div[1]
        employees = main_div[2]
    
    except:
        valuation = 'not found'
        price_hour = 'not found'
        employees = 'not found'
    
    clients = ''
    try:
        clients = soup.find("div", attrs={"class" : "field-name-clients"})
        clients =  ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
    except:
        try:
            clients = soup.find("div", attrs={"class" : "field c"})
            clients = ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
        except:
            print(f"CLIENTES NÃO ENCONTRADOS EM {name}")
            
    return {'name': name,
            'valuation': valuation,
            'hour_price': price_hour,
            'employees': employees,
            'score': score,
            'location': location,
            'clients': clients}

In [12]:
def collect_companies(lim_search: int = 1000):
    
    print('Accessing: ', clutch_url)
    browser.get(clutch_url)
    time.sleep(2)
    
    pages_limit = int(lim_search/20)
    pages_founded = get_total_pages_at(browser)
    pages = min(pages_founded,pages_limit)
    
    print('Get: ', str(pages), 'pages')
    
    for i in range(1,pages):
        
        print('Accessing page: ', i)
        browser.get(f"https://clutch.co/it-services/outsourcing?page={i}")
        time.sleep(1)
        
        reference_list = find_url_references_at(browser)
        print('Find ', len(reference_list), 'references')
        
        clutch_df = collect_data_from(reference_list)
        clutch_df.to_csv(companies_path, index=False)
    
    return clutch_df

In [13]:
collect_companies()

get_companies_csv()

Accessing:  https://clutch.co/it-services/outsourcing
Get:  50 pages
Accessing page:  1
Find  20 references
CLIENTES NÃO ENCONTRADOS EM delaPlex
CLIENTES NÃO ENCONTRADOS EM Avenga
CLIENTES NÃO ENCONTRADOS EM TechMagic
CLIENTES NÃO ENCONTRADOS EM Skelia
Accessing page:  2
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Alpacked
Accessing page:  3
Find  20 references
Accessing page:  4
Find  20 references
CLIENTES NÃO ENCONTRADOS EM CHI Software
CLIENTES NÃO ENCONTRADOS EM Oneture Technologies
CLIENTES NÃO ENCONTRADOS EM Evojam
Accessing page:  5
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Aureate Labs
CLIENTES NÃO ENCONTRADOS EM Principle Co, Ltd.
CLIENTES NÃO ENCONTRADOS EM Resolute Software
Accessing page:  6
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Asamby Consulting
CLIENTES NÃO ENCONTRADOS EM Cut2Code
CLIENTES NÃO ENCONTRADOS EM OneClick IT Consultancy Pvt. Ltd.
CLIENTES NÃO ENCONTRADOS EM Giraffe Studio
Accessing page:  7
Find  20 references
CLIENTES NÃO ENCONTRADOS EM P

CLIENTES NÃO ENCONTRADOS EM Locuz
CLIENTES NÃO ENCONTRADOS EM Braineering IT Solutions
Accessing page:  9
Find  20 references
CLIENTES NÃO ENCONTRADOS EM IT Svit
CLIENTES NÃO ENCONTRADOS EM Kyos
CLIENTES NÃO ENCONTRADOS EM Code Inspiration
Accessing page:  10
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Fly On The Cloud
CLIENTES NÃO ENCONTRADOS EM Binary Republik
CLIENTES NÃO ENCONTRADOS EM Euvic
CLIENTES NÃO ENCONTRADOS EM VirtusaPolaris
CLIENTES NÃO ENCONTRADOS EM Langate Corp
Accessing page:  11
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Welcome Networks
CLIENTES NÃO ENCONTRADOS EM InCrys
CLIENTES NÃO ENCONTRADOS EM ITGix
CLIENTES NÃO ENCONTRADOS EM HashRoot Ltd
CLIENTES NÃO ENCONTRADOS EM Insitu S2 Tikshuv LTD
CLIENTES NÃO ENCONTRADOS EM R&D Solutions
Accessing page:  12
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Natsoft Corporation
CLIENTES NÃO ENCONTRADOS EM ITGLOBAL.COM
CLIENTES NÃO ENCONTRADOS EM Israel IT
CLIENTES NÃO ENCONTRADOS EM TechnoGlobal Team Inc
CLIENTES 

Accessing page:  16
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Andes Digital
CLIENTES NÃO ENCONTRADOS EM 112HUB
CLIENTES NÃO ENCONTRADOS EM Epsilon
CLIENTES NÃO ENCONTRADOS EM boredDevOps
CLIENTES NÃO ENCONTRADOS EM Scalex Technology Solutions
CLIENTES NÃO ENCONTRADOS EM Brevitaz Systems
CLIENTES NÃO ENCONTRADOS EM IT Respublica
CLIENTES NÃO ENCONTRADOS EM Chrisranjana.com data engineers ETL developers
Accessing page:  17
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Cloudica
CLIENTES NÃO ENCONTRADOS EM Cloud Analogy
CLIENTES NÃO ENCONTRADOS EM Intertec International
CLIENTES NÃO ENCONTRADOS EM Cleverbit
CLIENTES NÃO ENCONTRADOS EM AquaOrange (ThaiPC Support)
CLIENTES NÃO ENCONTRADOS EM Smartym Pro
CLIENTES NÃO ENCONTRADOS EM MSPAssist
Accessing page:  18
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Creative People
CLIENTES NÃO ENCONTRADOS EM Stridely Solutions
CLIENTES NÃO ENCONTRADOS EM Techmango Technology Services Private Limited
CLIENTES NÃO ENCONTRADOS EM AACROM
CLIENTE

CLIENTES NÃO ENCONTRADOS EM The One Technologies
Accessing page:  23
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Zimmic
CLIENTES NÃO ENCONTRADOS EM OPGK Software
CLIENTES NÃO ENCONTRADOS EM 100 CRM
CLIENTES NÃO ENCONTRADOS EM Spaceport
Accessing page:  24
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Veloxcore
CLIENTES NÃO ENCONTRADOS EM Atharva System
CLIENTES NÃO ENCONTRADOS EM Sekurno
CLIENTES NÃO ENCONTRADOS EM PieSoft
CLIENTES NÃO ENCONTRADOS EM MattsenKumar LLC
CLIENTES NÃO ENCONTRADOS EM Trilight Security
Accessing page:  25
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Amalgama
CLIENTES NÃO ENCONTRADOS EM Green Apex Technolabs
CLIENTES NÃO ENCONTRADOS EM Sudofy
CLIENTES NÃO ENCONTRADOS EM VirtusLab
CLIENTES NÃO ENCONTRADOS EM Sattrix
Accessing page:  26
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Persistent Systems
CLIENTES NÃO ENCONTRADOS EM DS Stream
CLIENTES NÃO ENCONTRADOS EM Innovation Development HUB
CLIENTES NÃO ENCONTRADOS EM Grafixoft
CLIENTES NÃO ENCONTRAD

Accessing page:  30
Find  20 references
CLIENTES NÃO ENCONTRADOS EM VTI Cloud
CLIENTES NÃO ENCONTRADOS EM GRIP Software Solutions
CLIENTES NÃO ENCONTRADOS EM Logix Technology
CLIENTES NÃO ENCONTRADOS EM Geek-Solutions
CLIENTES NÃO ENCONTRADOS EM 1902 Software Development Corporation
CLIENTES NÃO ENCONTRADOS EM VISUAL BI SOLUTIONS
Accessing page:  31
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Diya
CLIENTES NÃO ENCONTRADOS EM OSworkshop
CLIENTES NÃO ENCONTRADOS EM Omnist Techhub Solutions Private Limited
CLIENTES NÃO ENCONTRADOS EM Pofco Solution Consultancy & Services
CLIENTES NÃO ENCONTRADOS EM CiELO Technologies
CLIENTES NÃO ENCONTRADOS EM Arbutus
Accessing page:  32
Find  20 references
CLIENTES NÃO ENCONTRADOS EM eagerWorks
CLIENTES NÃO ENCONTRADOS EM Mannya Techno Solutions Inc
CLIENTES NÃO ENCONTRADOS EM Dataeaze Systems
CLIENTES NÃO ENCONTRADOS EM goodylabs
CLIENTES NÃO ENCONTRADOS EM Intelligence Partner
Accessing page:  33
Find  20 references
CLIENTES NÃO ENCONTRADOS EM AFo

Accessing page:  37
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Prolitus Technologies
CLIENTES NÃO ENCONTRADOS EM e-Lietuva
CLIENTES NÃO ENCONTRADOS EM Eternal Web Pvt. Ltd.
CLIENTES NÃO ENCONTRADOS EM Baltic Amadeus
CLIENTES NÃO ENCONTRADOS EM JDV Technologies
CLIENTES NÃO ENCONTRADOS EM Timvero
CLIENTES NÃO ENCONTRADOS EM BULPROS
CLIENTES NÃO ENCONTRADOS EM SISU Technologies
CLIENTES NÃO ENCONTRADOS EM Shaligram Infotech
CLIENTES NÃO ENCONTRADOS EM FastLogic
Accessing page:  38
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Perfsys
CLIENTES NÃO ENCONTRADOS EM Softwarium
CLIENTES NÃO ENCONTRADOS EM Cloud Solutions
CLIENTES NÃO ENCONTRADOS EM Algoscale Technologies, Inc.
CLIENTES NÃO ENCONTRADOS EM MindQuad Solutions Pvt. Ltd.
CLIENTES NÃO ENCONTRADOS EM Appliscale
CLIENTES NÃO ENCONTRADOS EM IVALUEPLUS SERVICES PVT LTD
Accessing page:  39
Find  20 references
CLIENTES NÃO ENCONTRADOS EM NextBits
CLIENTES NÃO ENCONTRADOS EM MANGOSOFT
CLIENTES NÃO ENCONTRADOS EM GTC
CLIENTES NÃO ENC

CLIENTES NÃO ENCONTRADOS EM ISK Global
CLIENTES NÃO ENCONTRADOS EM Alphosys Technologies Pvt Ltd
Accessing page:  44
Find  20 references
CLIENTES NÃO ENCONTRADOS EM DeepInspire
CLIENTES NÃO ENCONTRADOS EM BRAIN:IT
CLIENTES NÃO ENCONTRADOS EM Hexis d.o.o.
CLIENTES NÃO ENCONTRADOS EM Acropolium
CLIENTES NÃO ENCONTRADOS EM Koukio Solutions
CLIENTES NÃO ENCONTRADOS EM Zeliant Infoserve Pvt. Ltd.
CLIENTES NÃO ENCONTRADOS EM Shree AGT Multimedia
CLIENTES NÃO ENCONTRADOS EM SiteGuarding CY Ltd.
Accessing page:  45
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Cymune Cyber Security
CLIENTES NÃO ENCONTRADOS EM RMRF
CLIENTES NÃO ENCONTRADOS EM Tech Mitraa
CLIENTES NÃO ENCONTRADOS EM GenuineBee
CLIENTES NÃO ENCONTRADOS EM UpScale Ninjas
CLIENTES NÃO ENCONTRADOS EM Rudder Analytics
CLIENTES NÃO ENCONTRADOS EM QualityWorks Consulting Group
CLIENTES NÃO ENCONTRADOS EM Sangatek
CLIENTES NÃO ENCONTRADOS EM DefragLabs
CLIENTES NÃO ENCONTRADOS EM Lemlock
Accessing page:  46
Find  20 references
CLIENTE

CLIENTES NÃO ENCONTRADOS EM Synapse Team


,clients,employees,hour_price,location,name,score,valuation
0,Ticketbooth Boatsetter Trumedia Blade IBSS Cu...,10 - 49,$25 - $49 / hr,"Saltillo, Mexico",ClickIT Smart Technologies,4.9,"$5,000+"
1,NaN,250 - 999,$25 - $49 / hr,"Atlanta, GA",delaPlex,5.0,"$25,000+"
2,"RTL Hrvatska, Pro-bit, Dudubags, Veliki Odmor",2 - 9,$50 - $99 / hr,"Pula, Croatia",Sysbee,5.0,"$1,000+"
3,"Fortune 500 companies, Gogo, Fluke, TuneIn, H...","1,000 - 9,999",$50 - $99 / hr,"Lviv, Ukraine",N-iX,4.9,"$100,000+"
4,NaN,"1,000 - 9,999",$50 - $99 / hr,"Rochelle Park, NJ",Avenga,4.8,"$50,000+"
...,...,...,...,...,...,...,...
1195,NaN,10 - 49,$25 - $49 / hr,"Kharkiv, Ukraine",Space Whale,5.0,"$25,000+"
1196,"TD Ameritrade, Bridgestone Firestone, Dell EM...",10 - 49,$100 - $149 / hr,"New York, NY",impakt Advisors,5.0,"$10,000+"
1197,,10 - 49,$25 - $49 / hr,"Stockholm, Sweden",Chimplie,4.5,"$5,000+"
1198,NaN,2 - 9,< $25 / hr,"Jaipur, India",Kaptcha Softwares,5.0,"$1,000+"


In [18]:
df_clean = get_companies_csv().drop_duplicates(subset=['name'])
df_clean.to_csv(companies_path, index=False) 
df_clean

,clients,employees,hour_price,location,name,score,valuation
0,Ticketbooth Boatsetter Trumedia Blade IBSS Cu...,10 - 49,$25 - $49 / hr,"Saltillo, Mexico",ClickIT Smart Technologies,4.9,"$5,000+"
1,NaN,250 - 999,$25 - $49 / hr,"Atlanta, GA",delaPlex,5.0,"$25,000+"
2,"RTL Hrvatska, Pro-bit, Dudubags, Veliki Odmor",2 - 9,$50 - $99 / hr,"Pula, Croatia",Sysbee,5.0,"$1,000+"
3,"Fortune 500 companies, Gogo, Fluke, TuneIn, H...","1,000 - 9,999",$50 - $99 / hr,"Lviv, Ukraine",N-iX,4.9,"$100,000+"
4,NaN,"1,000 - 9,999",$50 - $99 / hr,"Rochelle Park, NJ",Avenga,4.8,"$50,000+"
...,...,...,...,...,...,...,...
1195,NaN,10 - 49,$25 - $49 / hr,"Kharkiv, Ukraine",Space Whale,5.0,"$25,000+"
1196,"TD Ameritrade, Bridgestone Firestone, Dell EM...",10 - 49,$100 - $149 / hr,"New York, NY",impakt Advisors,5.0,"$10,000+"
1197,,10 - 49,$25 - $49 / hr,"Stockholm, Sweden",Chimplie,4.5,"$5,000+"
1198,NaN,2 - 9,< $25 / hr,"Jaipur, India",Kaptcha Softwares,5.0,"$1,000+"


In [19]:
df_clean.sort_values(by=['score'], ascending=False)

,clients,employees,hour_price,location,name,score,valuation
373,"Nimble inc, GoodData, Open source",2 - 9,$25 - $49 / hr,"Shlapanice, Brno-Country District, Czech Republic",abstractR,not found,"$1,000+"
955,NaN,"1,000 - 9,999",$25 - $49 / hr,"Sofia, Bulgaria",BULPROS,not found,"$5,000+"
854,NaN,50 - 249,$150 - $199 / hr,"Madrid, Spain",Intelligence Partner,not found,Undisclosed
460,"Qwintry, Citrus, StilyoApps",10 - 49,< $25 / hr,"Kyiv, Ukraine",SYSTEM ADMINS PRO,5.0,"$1,000+"
546,XEQUALS,2 - 9,$150 - $199 / hr,"Wellington, New Zealand",CLVR Cloud Native Solutions,5.0,"$10,000+"
...,...,...,...,...,...,...,...
924,"Center for Action and Contemplation, MTOPO Pa...",50 - 249,$25 - $49 / hr,"Coral Springs, FL",Prakash Software Solutions,4.0,"$10,000+"
525,"Foot Locker, Wild Birds Unlimited, Brookstone...","1,000 - 9,999",Undisclosed,"Plantation, FL","Chetu, Inc.",3.8,"$10,000+"
901,"CloudOn, Dropbox, Audi, USAID, Bueno, Microso...",50 - 249,$50 - $99 / hr,"Milpitas, CA",IQVIS,3.5,"$5,000+"
657,BNP Paribas; Airbus; Seoullimited; Kaesa; Giv...,2 - 9,$150 - $199 / hr,"Seoul, South Korea",Kairos,3.5,"$1,000+"


In [22]:
df_clean.sort_values(by=['valuation'], ascending=True)

,clients,employees,hour_price,location,name,score,valuation
946,"ABInbev, UN, US State Gov't, Bank of India",10 - 49,< $25 / hr,"Kampala, Uganda",Efficiencie,5.0,"$1,000+"
676,"ECOMPLY.io, Coding Blog, Stakeholder Club Inc...",2 - 9,$25 - $49 / hr,"Munich, Germany",Connect Platform,4.8,"$1,000+"
400,NaN,10 - 49,$50 - $99 / hr,"Wrocław, Poland",Fly On The Cloud,5.0,"$1,000+"
401,"Third Wave Network, Real Trade Imobiliare",2 - 9,$25 - $49 / hr,"Brasov, Romania",BKHosting,5.0,"$1,000+"
1108,NaN,2 - 9,< $25 / hr,"Ahmedabad, India",Tech Mitraa,5.0,"$1,000+"
...,...,...,...,...,...,...,...
463,"selectspecs.com, ab.ua, autocentre.ua",10 - 49,$25 - $49 / hr,"Irpin, Ukraine",CoreQ,5.0,Undisclosed
1080,NaN,10 - 49,$25 - $49 / hr,"L'viv, Ukraine",DeepInspire,5.0,Undisclosed
456,"EY (former Ernst&Young), Tele2, Effective Cov...",50 - 249,$25 - $49 / hr,"New York City, NY",Azoft,5.0,Undisclosed
1070,- http://www.olbg.com/ - http://www.viihealth...,10 - 49,$25 - $49 / hr,"Bucharest, Romania",Devodrome,4.6,Undisclosed


In [21]:
df_clean.sort_values(by=['employees'], ascending=False)

,clients,employees,hour_price,location,name,score,valuation
835,"12Labels Inc, Primetime Ventures Pty Ltd, Eyw...",Freelancer,< $25 / hr,"Solapur, India",CloudLab.in,4.9,"$1,000+"
846,I will add this later,Freelancer,$25 - $49 / hr,"Kolkata, India",ELOGICSOFT.COM,5.0,"$1,000+"
1170,"Investment Banking, Capital Markets, Financial...",Freelancer,$100 - $149 / hr,"London, United Kingdom",The London Software Testing Company,5.0,"$100,000+"
744,NaN,Freelancer,$50 - $99 / hr,"Warsaw, Poland",Varstadt,5.0,"$1,000+"
1051,NaN,50 - 249,Undisclosed,"Atlanta, GA",Kratikal Tech Pvt Ltd,4.9,Undisclosed
...,...,...,...,...,...,...,...
388,"Samsung, Unite Private Networks, Windstream, C...","1,000 - 9,999",$50 - $99 / hr,"Pittsford, NY",Netsmartz LLC,4.9,"$25,000+"
384,"Panasonic, Henry Schein Practice Solutions, B...","1,000 - 9,999",$100 - $149 / hr,"Austin, TX",SoftServe,4.8,"$50,000+"
955,NaN,"1,000 - 9,999",$25 - $49 / hr,"Sofia, Bulgaria",BULPROS,not found,"$5,000+"
376,"Marlabs works with global enterprises, mid-si...","1,000 - 9,999",Undisclosed,"Piscataway, NJ",Marlabs Inc,4.0,"$100,000+"
